In [248]:
# Import the relevent packages
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [258]:
raw_data = pd.read_csv("RawData.csv")
raw_data['Date'] = pd.to_datetime(raw_data['Date']) # Modify the 'Date' column to 'datetime' format to be able to select years

In [259]:
# Split the sets. I used 2013 for cross validation. 2014 and 2015 for training set 
training_set = raw_data[raw_data['Date'].dt.year.isin(range(2014, 2016))]
cross_val_set = raw_data[raw_data['Date'].dt.year == 2013] 
test_set = raw_data[raw_data['Date'].dt.year.isin(range(2016, 2020))]

In [260]:
# Pre-process the training set
# Date and the name of the player are irrelevent to the Brownlow votes, and slowed down the performance, so I removed them. 

training_set = training_set.drop("Date", axis = 1)
training_set = training_set.drop("Name", axis = 1)
training_set = pd.get_dummies(training_set) # Convert categorical variables to indicators 

x_train = training_set.drop("Brownlow Votes",axis = 1) # Initialise the input parameters
y_train = training_set["Brownlow Votes"] # Initialise the target parameter

# Pre-process the cross validation set
cross_val_set = cross_val_set.drop("Date", axis = 1)
cross_val_set = cross_val_set.drop("Name", axis = 1)
cross_val_set = pd.get_dummies(cross_val_set) # Convert categorical variables to indicators 
x_cross_val =  cross_val_set.drop("Brownlow Votes",axis = 1) # Initialise the input parameters
y_cross_val = cross_val_set["Brownlow Votes"] # Initialise the target parameter
y_pred = log_reg.predict(x_cross_val) 

# Pre-process the test data set
test_set = test_set.drop("Date", axis = 1)
test_set = test_set.drop("Name", axis = 1)
test_set = pd.get_dummies(test_set) # Convert categorical variables to indicators 
x_test =  test_set.drop("Brownlow Votes",axis = 1) # Initialise the input parameters
y_test = test_set["Brownlow Votes"] # Initialise the target parameter
y_pred_test = log_reg.predict(x_test_set) 


In [261]:
# Logistic Regression
log_reg = LogisticRegression(solver='sag', max_iter = 5000)
log_reg.fit(x_train, y_train) # fit the parameters in to the model

LogisticRegression(max_iter=5000, solver='sag')

In [262]:
# Test the accuracy of the model using the cross validation set 
print("This model is {:.2f}% accurate".format(metrics.accuracy_score(y_cross_val, y_pred)*100.0))


This model is 94.01% accurate


In [304]:
# Predict on the test set
# Test the accuracy of the model
print("This model is {:.2f}% accurate".format(metrics.accuracy_score(y_test, y_pred_test)*100.0))

# Example: predict the winner of Brownlow medal for 2019 season
test_set_updated = raw_data[raw_data['Date'].dt.year.isin(range(2016, 2020))] # re-read the test set to include Name and Date
test_set_updated.insert(1,'Predicted Brownlow', y_pred_test) # add the predictions to the data
test_2019 = test_set_updated[test_set_updated['Date'].dt.year == 2019] # extract 2019 data 
res = test_2019.groupby('Name')['Predicted Brownlow'].sum().to_frame(name = 'Total Brownlow').reset_index() # group by players name to calculate the sum of brownlow in the whole season

print("The predicted winners of 2019 season are:")
res.nlargest(3, 'Total Brownlow')

This model is 94.18% accurate
The predicted winners of 2019 season are:


,Name,Total Brownlow
82,Brodie Grundy,27.0
493,Patrick Dangerfield,24.0
178,Dustin Martin,21.0
